In [1]:
import pandas as pd

In [14]:
pav_camel = pd.read_csv('pav_camel_v_1.csv', index_col=0, dtype={'bin':'str'})
pav_camel.head(2)

,bin,full_add,point,precision
49,140002415,"г. Павлодар,ПРОМЫШЛЕННАЯ ЗОНА ЦЕНТРАЛЬНАЯ,406",76.907556 52.30741,street
81,140006646,"обл. Павлодарская,с. Теренколь,Тәуелсіздік,88-2",76.108334 53.064051,street


In [28]:
pav_upper = pd.read_csv('pav_upper_v_1.csv', index_col=0,dtype={'bin':'str'})
pav_upper.head(2)

,bin,full_add,point,precision
74,140006180,"ПАВЛОДАР,Торайгырова,56",76.95143 52.297299,exact
133,240004574,"ПАВЛОДАР,ул. Естая,д. 81",76.960323 52.280652,exact


In [36]:
pav_upper['len'] = pav_upper['bin'].str.len()

In [37]:
pav_upper['len'].value_counts()

12    1863
Name: len, dtype: int64

In [33]:
pav_upper.loc[pav_upper['len'] == 9, 'bin'] = '000'+pav_upper['bin']

In [41]:
pavlodar_full = pd.concat([pav_camel, pav_upper]).reset_index(drop=True)

In [43]:
pavlodar_full.dropna(subset=['point'], inplace=True)

In [55]:
pavlodar_full[['lat','lon']] = pavlodar_full['point'].str.split(' ',1,expand=True)

In [61]:
pavlodar_full[['lat','lon']] = pavlodar_full[['lat','lon']].astype('float')

In [56]:
pavlodar_full

,bin,full_add,point,precision,len,lat,lon
0,000140002415,"г. Павлодар,ПРОМЫШЛЕННАЯ ЗОНА ЦЕНТРАЛЬНАЯ,406",76.907556 52.30741,street,12,76.907556,52.30741
1,000140006646,"обл. Павлодарская,с. Теренколь,Тәуелсіздік,88-2",76.108334 53.064051,street,12,76.108334,53.064051
3,000240005255,"обл. Павлодарская,г. Павлодар,Гагарина,16",76.947477 52.259892,exact,12,76.947477,52.259892
4,000240006987,"обл. Павлодарская,г. Павлодар,Лермонтова,109",76.964078 52.291009,exact,12,76.964078,52.291009
5,000313551227,"обл. Павлодарская,г. Павлодар,Павлова,36/3",76.974714 52.289825,exact,12,76.974714,52.289825
...,...,...,...,...,...,...,...
6300,991140005992,"ПАВЛОДАР,ул. Сураганова, д. 19",76.972891 52.2933,number,12,76.972891,52.2933
6301,991141005511,"ПАВЛОДАР,Торайгырова,79",76.953532 52.296578,exact,12,76.953532,52.296578
6302,991240003553,"ПАВЛОДАР,Черепанова,10",76.953424 52.300361,exact,12,76.953424,52.300361
6303,991240005292,"ПАВЛОДАР,Пахомова,102",76.962722 52.252042,exact,12,76.962722,52.252042


In [57]:
import h3
def create_hexagon_clusters(data, low_level_res = 9):
    """low_level_res & high_level_res размеры гексагонов нижнего и верхнего уровня"""
    #создание кластера нижнего уровня
    data['cluster'] = [h3.geo_to_h3(row['lon'], row['lat'], low_level_res) for idx, row in data.iterrows()]
    return data

In [63]:
pavlodar_full = create_hexagon_clusters(pavlodar_full)

In [65]:
pavlodar_full

,bin,full_add,point,precision,len,lat,lon,cluster
0,000140002415,"г. Павлодар,ПРОМЫШЛЕННАЯ ЗОНА ЦЕНТРАЛЬНАЯ,406",76.907556 52.30741,street,12,76.907556,52.307410,89215910a13ffff
1,000140006646,"обл. Павлодарская,с. Теренколь,Тәуелсіздік,88-2",76.108334 53.064051,street,12,76.108334,53.064051,89215d30837ffff
3,000240005255,"обл. Павлодарская,г. Павлодар,Гагарина,16",76.947477 52.259892,exact,12,76.947477,52.259892,89215902663ffff
4,000240006987,"обл. Павлодарская,г. Павлодар,Лермонтова,109",76.964078 52.291009,exact,12,76.964078,52.291009,8921590242fffff
5,000313551227,"обл. Павлодарская,г. Павлодар,Павлова,36/3",76.974714 52.289825,exact,12,76.974714,52.289825,8921590254bffff
...,...,...,...,...,...,...,...,...
6300,991140005992,"ПАВЛОДАР,ул. Сураганова, д. 19",76.972891 52.2933,number,12,76.972891,52.293300,89215902543ffff
6301,991141005511,"ПАВЛОДАР,Торайгырова,79",76.953532 52.296578,exact,12,76.953532,52.296578,892159024afffff
6302,991240003553,"ПАВЛОДАР,Черепанова,10",76.953424 52.300361,exact,12,76.953424,52.300361,892159024a3ffff
6303,991240005292,"ПАВЛОДАР,Пахомова,102",76.962722 52.252042,exact,12,76.962722,52.252042,892159022c7ffff


In [68]:
#company information from statistics comitee
comstat_df = pd.read_csv('~/Desktop/ofd/data/company_info_full.csv', index_col=0, dtype={'БИН':str})
comstat_df.head(2)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,БИН,Толық атауы,Полное наименование,Дата регистрации,ОКЭД,Негізгі қызмет түрінің атауы,Наименование основного вида деятельности,Втор.ОКЭД,КРП,ККЖ атауы,Наименование КРП,КАТО,Елді мекеннің атауы,Наименование населенного пункта,Юридический адрес,"Басшының ТАӘ, ФИО руководителя",Юридическая форма
0,040940010501,"""В 2 В COMPANY"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТ...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2004-09-30,47191,Сауда алаңы 2000 ш.м-ден кем сауда объектілері...,Прочая розничная торговля в неспециализированн...,NaN,105,Шағын кәсіпорындар (<= 5),Малые предприятия (<= 5),351011100,"ҚАРАҒАНДЫ ОБЛЫСЫ, ҚАРАҒАНДЫ Қ.Ә., ҚАРАҒАНДЫ Қ....","ҚАРАҒАНДЫ ОБЛЫСЫ, ҚАРАҒАНДЫ Қ.Ә., ҚАРАҒАНДЫ Қ....","МИКРОРАЙОН МАМРАЕВА, 41, КВ 86",СЕРОВ ДМИТРИЙ ИВАНОВИЧ,Юрлицо
1,131140017879,"""ELITE CONSTRUCTION"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ С...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2013-11-20,22231,Құрылыста пайдаланылатын пластмасса бұйымдарын...,"Производство пластмассовых изделий, используем...",NaN,105,Шағын кәсіпорындар (<= 5),Малые предприятия (<= 5),791510000,"ШЫМКЕНТ Қ., ЕҢБЕКШІ АУДАНЫ","ШЫМКЕНТ Қ., ЕҢБЕКШІ АУДАНЫ","УЛИЦА КОЛБАЕВА, Д.25",ТАСТАНБЕКОВ ЕРЛАН ӨТЖАНҰЛЫ,Юрлицо


In [70]:
pavlodar_full_comstat = pavlodar_full.merge(comstat_df, how='left',left_on='bin',right_on='БИН')

In [72]:
ofd_data = pd.read_csv('ofd_mean_count_sum.csv', index_col=0, dtype={'KkmsRetailPlaceOrgCatalog_orgInn':'str'})
ofd_data.head(2)

,KkmsRetailPlaceOrgCatalog_orgInn,sum,sum.1,sum.2
0,000103501685,5180.000000,5,25900.0
1,000105650066,1910.906667,225,429954.0


In [73]:
ofd_data.rename(columns={'KkmsRetailPlaceOrgCatalog_orgInn':'bin', 'sum':'mean','sum.1':'count', 'sum.2':'sum'}, inplace=True)

In [76]:
pavlodar_full_comstat_ofd = pavlodar_full_comstat.merge(ofd_data,how='left',on='bin')

In [78]:
pavlodar_full_comstat_ofd.columns

Index(['bin', 'full_add', 'point', 'precision', 'len', 'lat', 'lon', 'cluster',
       'БИН', 'Толық атауы', 'Полное наименование ', 'Дата регистрации',
       'ОКЭД', 'Негізгі қызмет түрінің атауы',
       'Наименование основного вида деятельности', 'Втор.ОКЭД', 'КРП',
       'ККЖ атауы', 'Наименование КРП', 'КАТО', 'Елді мекеннің атауы',
       'Наименование населенного пункта', 'Юридический адрес',
       'Басшының ТАӘ, ФИО руководителя', 'Юридическая форма', 'mean', 'count',
       'sum'],
      dtype='object')

In [80]:
pavlodar_full_comstat_ofd = pavlodar_full_comstat_ofd[['bin','lat','lon','cluster','ОКЭД','Наименование основного вида деятельности',
                          'Наименование КРП', 'Юридическая форма', 'mean', 'count','sum']]

In [83]:
pavlodar_full_comstat_ofd.to_csv('pavlodar_heatmap.sample.csv')



In [110]:
pavlodar_full_comstat_ofd

,bin,lat,lon,cluster,ОКЭД,Наименование основного вида деятельности,Наименование КРП,Юридическая форма,mean,count,sum,coords,in_pavlodar
0,000140002415,76.907556,52.307410,89215910a13ffff,46751.0,Оптовая торговля химическими веществами и хими...,Малые предприятия (21-30),Юрлицо,17825.000000,2.0,35650.0,"{'type': 'Point', 'coordinates': [52.30741, 76...",True
1,000140006646,76.108334,53.064051,89215d30837ffff,86230.0,Стоматологическая деятельность,Малые предприятия (<= 5),Юрлицо,5250.000000,4.0,21000.0,"{'type': 'Point', 'coordinates': [53.064051, 7...",False
2,000240005255,76.947477,52.259892,89215902663ffff,86210.0,Общая врачебная практика,Малые предприятия (31-40),Юрлицо,21745.934211,76.0,1652691.0,"{'type': 'Point', 'coordinates': [52.259892, 7...",True
3,000240006987,76.964078,52.291009,8921590242fffff,86210.0,Общая врачебная практика,Малые предприятия (11-15),Юрлицо,2620.421053,190.0,497880.0,"{'type': 'Point', 'coordinates': [52.291009, 7...",True
4,000313551227,76.974714,52.289825,8921590254bffff,47241.0,"Розничная торговля хлебобулочными, мучными и с...",Малые предприятия (<= 5),ИП,338.333333,9.0,3045.0,"{'type': 'Point', 'coordinates': [52.289825, 7...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6242,991140005992,76.972891,52.293300,89215902543ffff,94990.0,"Деятельность прочих общественных организаций, ...",Малые предприятия (6 - 10),Юрлицо,28445.000000,1.0,28445.0,"{'type': 'Point', 'coordinates': [52.2933, 76....",True
6243,991141005511,76.953532,52.296578,892159024afffff,65122.0,Негосударственное страхование ущерба,Малые предприятия (<= 5),Филиал,3799.285714,63.0,239355.0,"{'type': 'Point', 'coordinates': [52.296578, 7...",True
6244,991240003553,76.953424,52.300361,892159024a3ffff,68201.0,Аренда и управление собственной недвижимостью,Малые предприятия (11-15),Юрлицо,40349.000000,5.0,201745.0,"{'type': 'Point', 'coordinates': [52.300361, 7...",True
6245,991240005292,76.962722,52.252042,892159022c7ffff,46360.0,"Оптовая торговля сахаром, шоколадом и сахарист...",Малые предприятия (51-100),Юрлицо,6147.618919,1110.0,6823857.0,"{'type': 'Point', 'coordinates': [52.252042, 7...",True


In [111]:
pavlodar_full_comstat_ofd['lon'] = pavlodar_full_comstat_ofd['lon'].astype('float')
pavlodar_full_comstat_ofd['lat'] = pavlodar_full_comstat_ofd['lat'].astype('float')

<ipython-input-111-11318a0ef884>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pavlodar_full_comstat_ofd['lon'] = pavlodar_full_comstat_ofd['lon'].astype('float')
<ipython-input-111-11318a0ef884>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pavlodar_full_comstat_ofd['lat'] = pavlodar_full_comstat_ofd['lat'].astype('float')


In [116]:
pavlodar_full_comstat_ofd['coords'] = list(zip(pavlodar_full_comstat_ofd['lon'],pavlodar_full_comstat_ofd['lat']))

<ipython-input-116-27c3b07b205d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pavlodar_full_comstat_ofd['coords'] = list(zip(pavlodar_full_comstat_ofd['lon'],pavlodar_full_comstat_ofd['lat']))


In [117]:
pavlodar_full_comstat_ofd['coords'] = pavlodar_full_comstat_ofd['coords'].apply(Point)

<ipython-input-117-7de84fafd938>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pavlodar_full_comstat_ofd['coords'] = pavlodar_full_comstat_ofd['coords'].apply(Point)


In [118]:
pavlodar_full_comstat_ofd['in_pavlodar'] = pavlodar_full_comstat_ofd['coords'].map(lambda x: boolean_point_in_polygon(x, polygon))

<ipython-input-118-0c9b7e6c08f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pavlodar_full_comstat_ofd['in_pavlodar'] = pavlodar_full_comstat_ofd['coords'].map(lambda x: boolean_point_in_polygon(x, polygon))


In [119]:
pavlodar_full_comstat_ofd['in_pavlodar'].value_counts()

True     4826
False    1421
Name: in_pavlodar, dtype: int64

In [120]:
pavlodar_full_comstat_ofd[pavlodar_full_comstat_ofd['in_pavlodar'] == True].to_csv('pavlodar_heatmap.sample_1.csv')

In [125]:
pavlodar_full_comstat_true = pavlodar_full_comstat_ofd[pavlodar_full_comstat_ofd['in_pavlodar'] == True].drop(['coords','in_pavlodar'],axis=1)

In [127]:
pavlodar_full_comstat_true.to_csv('pavlodar_heatmap_sample_true.csv')

In [104]:
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature

polygon = Polygon(
    [
        [
(52.2412177, 76.7474909),
(52.3130580, 76.8601008),
(52.3281675, 76.9047327),
(52.3285907, 76.9825745),
(52.3159964, 77.0187159),
(52.2420586, 77.0509882),
(52.2218723, 77.0152826),
(52.1941012, 77.0303888),
(52.1621009, 76.9466181),
(52.1528333, 76.8216486),
(52.1776829, 76.7811365),
(52.2412177, 76.7474909),
        ]
    ]
)

In [87]:
polygon_pav = pd.DataFrame(h3.polyfill_geojson(polygon,9))

In [88]:
polygon_pav.rename(columns={0: "hex_info"}, inplace=True)

In [89]:
polygon_pav

,hex_info
0,89215911b4bffff
1,892159112cfffff
2,8921591ad67ffff
3,8921590237bffff
4,89215903513ffff
...,...
2328,892159004c3ffff
2329,89215911223ffff
2330,89215911aafffff
2331,892159026d3ffff


In [ ]:
data['in_astana'] = data['coords'].map(lambda x: boolean_point_in_polygon(x, polygon))

In [128]:
pavlodar_full_comstat_true

,bin,lat,lon,cluster,ОКЭД,Наименование основного вида деятельности,Наименование КРП,Юридическая форма,mean,count,sum
0,000140002415,76.907556,52.307410,89215910a13ffff,46751.0,Оптовая торговля химическими веществами и хими...,Малые предприятия (21-30),Юрлицо,17825.000000,2.0,35650.0
2,000240005255,76.947477,52.259892,89215902663ffff,86210.0,Общая врачебная практика,Малые предприятия (31-40),Юрлицо,21745.934211,76.0,1652691.0
3,000240006987,76.964078,52.291009,8921590242fffff,86210.0,Общая врачебная практика,Малые предприятия (11-15),Юрлицо,2620.421053,190.0,497880.0
4,000313551227,76.974714,52.289825,8921590254bffff,47241.0,"Розничная торговля хлебобулочными, мучными и с...",Малые предприятия (<= 5),ИП,338.333333,9.0,3045.0
5,000340000733,76.957835,52.296759,892159024afffff,47789.0,Прочая розничная торговля в специализированных...,Малые предприятия (<= 5),Юрлицо,18372.562500,16.0,293961.0
...,...,...,...,...,...,...,...,...,...,...,...
6242,991140005992,76.972891,52.293300,89215902543ffff,94990.0,"Деятельность прочих общественных организаций, ...",Малые предприятия (6 - 10),Юрлицо,28445.000000,1.0,28445.0
6243,991141005511,76.953532,52.296578,892159024afffff,65122.0,Негосударственное страхование ущерба,Малые предприятия (<= 5),Филиал,3799.285714,63.0,239355.0
6244,991240003553,76.953424,52.300361,892159024a3ffff,68201.0,Аренда и управление собственной недвижимостью,Малые предприятия (11-15),Юрлицо,40349.000000,5.0,201745.0
6245,991240005292,76.962722,52.252042,892159022c7ffff,46360.0,"Оптовая торговля сахаром, шоколадом и сахарист...",Малые предприятия (51-100),Юрлицо,6147.618919,1110.0,6823857.0


In [136]:
pavlodar_full_hex = pavlodar_full_comstat_true.groupby('cluster').agg({'mean':'mean', 'count':'sum', 'sum':'sum'}).reset_index()

In [137]:
pavlodar_full_hex['hex_location'] = pavlodar_full_hex['cluster'].apply(lambda x: h3.h3_to_geo_boundary(x))

In [140]:
pavlodar_full_hex.dropna().to_csv('pavlodar_full_hex.csv')